In [ ]:
!pip install transformers datasets


In [ ]:
# import torch
# torch.cuda.is_available()#, torch.cuda.get_device_name(0)

True

In [ ]:
from datasets import Dataset,load_dataset

In [ ]:
#load datasets

In [ ]:
ds = load_dataset(
    "csv",
    data_files={"train":"train_en.tsv",
                "dev": "dev_test_en.tsv",
                "test": "test_en_labeled.tsv"
    },
    delimiter="\t"
)

In [ ]:
def encode_label(example):
    example["label"] = 0 if example["label"] == "OBJ" else 1
    return example
#clean ds
ds = ds.map(encode_label)
ds = ds.rename_column("label", "labels")
ds = ds.remove_columns("sentence_id")
ds = ds.remove_columns("solved_conflict")


In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(ds["train"])

,sentence,labels
0,A new variant that can evade vaccines and actually harm people will surface when the ruling class needs another lockdown to panic the unruly slaves into mass compliance out of fear.,1
1,"What the Levelling Up Bill is really is a Planning Bill in disguise, something the Committee made clear at the very beginning of their evidence sessions.",1
2,"If the natural level of economic recovery were long delayed, then all these measures would very soon fail in the total ruin of public credit.",1
3,"The tax power, in so many hands, is much less effective than it might be.",1
4,"A third formation of forces moving in a parallel manner to absorb the national income by extension of government is made up of practical reformers, idealists, good-government people, with or without any political theory.",1
5,"Therefore, taxes must be increased, first in order to provide as much public revenue as before, and then further increased to provide more revenue than before.",1
6,That the government are shoehorning what is left of their unpopular planning reforms into this Levelling Up legislation speaks to the vacuity of the ‘defining mission’.,1
7,A New England Journal of Medicine study has just confirmed what the data already has: that COVID-19 vaccines are giving people a new form of acquired immunodeficiency syndrome (AIDS).,0
8,"Taxes have risen to a point at which they begin to devour people’s possessions, and the taxpayer is wild for relief.",1
9,This makes reading the reports indoctrinated policy wonks write difficult to read because when they write about “immigrants” it is unclear if they actually mean lawfully present immigrants or if they are using the word as a euphemism for illegal aliens.,1


In [ ]:
model_checkpoint = 'distilbert/distilbert-base-uncased'
tokenizer_checkpoint = 'distilbert/distilbert-base-uncased'

In [ ]:
from transformers import (
    AutoTokenizer,
    DistilBertTokenizer
)

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained(tokenizer_checkpoint,truncation=True)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["sentence"],
        truncation=True,)

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [ ]:
# Then tokenize the 'sentence' column and remove it, keeping 'label' and the new tokenized columns.
tokenized_datasets = ds.map(tokenize_function, batched=True, num_proc=4, remove_columns=["sentence"])

In [ ]:
# caso nao precise agrupar
lm_datasets = tokenized_datasets

Funcao de perda utilizando Cross-Entropy

In [ ]:
from collections import Counter
import torch

label_counts = Counter(tokenized_datasets["train"]["labels"])
num_classes = len(label_counts)

# Example: dataset with labels 0 and 1
counts = [label_counts[i] for i in range(num_classes)]
total = sum(counts)

# Inverse frequency weighting
class_weights = [total / c for c in counts]

print("Class weights:", class_weights)

# Convert to tensor (move to CUDA later)
class_weights = torch.tensor(class_weights)


Class weights: [1.5601503759398496, 2.785234899328859]


In [ ]:
import torch
from torch import nn
from torch.nn import CrossEntropyLoss, MSELoss, BCEWithLogitsLoss
from transformers import DistilBertForSequenceClassification
from typing import Optional, Union
from transformers.modeling_outputs import SequenceClassifierOutput


class WeightedDistilBertForSequenceClassification(DistilBertForSequenceClassification):
    def __init__(self, config, class_weights=None):
        super().__init__(config)

        # Store weights (tensor or None)
        if class_weights is not None:
            self.class_weights = torch.tensor(class_weights, dtype=torch.float)
        else:
            self.class_weights = None


    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        labels: Optional[torch.LongTensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[SequenceClassifierOutput, tuple]:

        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        distilbert_output = self.distilbert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        hidden_state = distilbert_output[0]              # (batch_size, seq_len, dim)
        pooled_output = hidden_state[:, 0]              # (batch_size, dim)
        pooled_output = self.pre_classifier(pooled_output)
        pooled_output = nn.ReLU()(pooled_output)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:

            # Detect problem type (same logic as original)
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif labels.dtype in (torch.long, torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            # Regression
            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                if self.num_labels == 1:
                    loss = loss_fct(logits.squeeze(), labels.squeeze())
                else:
                    loss = loss_fct(logits, labels)

            # Single-label classification (this is the one we modify)
            elif self.config.problem_type == "single_label_classification":
                if self.class_weights is not None:
                    # Move class weights to the same device
                    class_weights = self.class_weights.to(logits.device)
                    loss_fct = CrossEntropyLoss(weight=class_weights)
                else:
                    loss_fct = CrossEntropyLoss()

                loss = loss_fct(
                    logits.view(-1, self.num_labels),
                    labels.view(-1)
                )

            # Multi-label classification - unchanged
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)

        if not return_dict:
            output = (logits,) + distilbert_output[1:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=distilbert_output.hidden_states,
            attentions=distilbert_output.attentions,
        )

In [ ]:
import torch
from torch import nn
from transformers import DistilBertForSequenceClassification
from transformers.modeling_outputs import SequenceClassifierOutput

class WeightedDistilBert(DistilBertForSequenceClassification):
    def __init__(self, config, class_weights=None):
        super().__init__(config)
        self.class_weights = class_weights

    def forward(self, *args, **kwargs):
        labels = kwargs.get("labels", None)

        # Remove labels so DistilBERT does NOT compute its own loss
        model_kwargs = {k: v for k, v in kwargs.items() if k != "labels"}

        # Call the original DistilBERT forward
        outputs = super().forward(*args, **model_kwargs)

        logits = outputs.logits
        loss = None

        if labels is not None:
            labels = labels.to(logits.device)
            loss_fct = nn.CrossEntropyLoss(
                weight=self.class_weights.to(logits.device)
                if self.class_weights is not None
                else None
            )
            loss = loss_fct(logits, labels)

        return SequenceClassifierOutput(
            loss = loss,
            logits = logits,
            hidden_states = outputs.hidden_states,
            attentions = outputs.attentions,
        )


Vamos criar as variáveis a partir do checkpoint do modelo

In [ ]:
from transformers import (
    AutoConfig,
    DistilBertConfig,
    DistilBertForSequenceClassification
)

In [ ]:


num_labels = len(set(lm_datasets["train"]["labels"]))

# config = DistilBertConfig.from_pretrained(model_checkpoint,
#                                     num_labels=num_labels,
#                                     class_weights=class_weights,
#                                     hidden_dropout_prob=0.1,
#                                     attention_probs_dropout_prob=0.1,
#                                     id2label={i: str(i) for i in range(num_labels)},
#                                     label2id={str(i): i for i in range(num_labels)}
#                                     )
#
model = WeightedDistilBertForSequenceClassification.from_pretrained(model_checkpoint,
    num_labels=num_labels,
    id2label={i: str(i) for i in range(num_labels)},
    label2id={str(i): i for i in range(num_labels)},
    class_weights=class_weights,   # ← vetor de pesos por classe
    # gamma=2.0
    )


# carregamento do checkpoint
# model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)

/tmp/ipython-input-2752834623.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.class_weights = torch.tensor(class_weights, dtype=torch.float)
Some weights of WeightedDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Definindo hiperparâmetros do treinamento

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
training_args = TrainingArguments(
    resume_from_checkpoint=False,
    output_dir="./checkpoint/",
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=2e-5,
    num_train_epochs=4,
    weight_decay=0.01,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    # fp16=True,
    # optim="adam",
    # gradient_accumulation_steps=1,
    load_best_model_at_end=True,
    report_to="none",
    # ,
    # logging_dir="./logs/",
    logging_steps=10,
    logging_strategy="steps",
    # seed=42,
    remove_unused_columns=True,
)

Classe de treino

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["dev"],
    data_collator=data_collator,

)

In [ ]:
print(model.state_dict()['distilbert.transformer.layer.0.attention.q_lin.weight'][0][:5])


tensor([-0.0024,  0.0224, -0.0207,  0.0318, -0.0102], device='cuda:0')


And we can train our model:

In [ ]:
import time
start = time.time()
trainer.train()
print (time.time() - start)

Epoch,Training Loss,Validation Loss
1,0.547800,0.522398
2,0.319500,0.486307
3,0.270100,0.625172
4,0.042600,0.666239


71.9787917137146


In [ ]:
print(model.state_dict()['distilbert.transformer.layer.0.attention.q_lin.weight'][0][:5])

pred = trainer.predict(tokenized_datasets["test"])
print(pred.predictions.shape)
print(pred.label_ids is not None)



tensor([-0.0025,  0.0222, -0.0210,  0.0312, -0.0100], device='cuda:0')


AttributeError: 'tuple' object has no attribute 'shape'

Analisando resultados

In [ ]:
from sklearn.metrics import classification_report

pred = trainer.predict(tokenized_datasets["test"])

logits = pred.predictions[0] if isinstance(pred.predictions, tuple) else pred.predictions

y_pred = logits.argmax(axis=-1)

y_true = pred.label_ids

print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           0       0.85      0.78      0.81       215
           1       0.54      0.66      0.59        85

    accuracy                           0.74       300
   macro avg       0.70      0.72      0.70       300
weighted avg       0.76      0.74      0.75       300



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

save_path = "/content/drive/MyDrive/Mestrado/Trabalho_LLM/Checkpoints_Distilbert"
os.makedirs(save_path, exist_ok=True)


In [ ]:
import shutil
import os

src = "/content/checkpoint/"
dst = "/content/drive/MyDrive/Mestrado/Trabalho_LLM/Checkpoints_Distilbert"

# create parent directory if needed
os.makedirs(os.path.dirname(dst), exist_ok=True)

# copy the entire folder
shutil.copytree(src, dst, dirs_exist_ok=True)

print("Checkpoint copied successfully!")


Checkpoint copied successfully!
